# **Valorisation des entreprises**

Dans ce travail nous allons se baser sur la dataset `Data_Finpack_V2` contenant les informations financières de plusieurs entreprises, pour développer un modèle de clustering pour les structurer en groupe, Les entreprises presentes dans chaque groupe s'appellent des comparables. 

### **Importation des packages**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from sklearn.preprocessing import  LabelEncoder
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import deepcopy
import pickle


### **Importation de la base de données**

In [ ]:
#Importation de la base de données
data = pd.read_excel( 'Data Finpack V2/Data_Finpack_V2.xlsx' )
print ('Taille de la base de donnes :', data.shape)

In [ ]:
#Les 5 premières lignes de la base de données
data.head()

In [ ]:
#Ne pas Supprimer  la colonne efcode
#data = data.drop(columns = 'Efcode', axis = 1)
#data.shape

In [ ]:
#Le nombre d'entreprise dans data
print (''' Le nombre d'entreprise dans data est : ''', data['Company Name'].value_counts().count())

### **Nettoyage et préparation des données**

#### **Duplicates handling**

In [ ]:
#Nombre de colonnes duppliquées
print ('le nombre de colonnes duppliquées : ', data.T.duplicated().sum())

In [ ]:
#Supression des colonnes duppliqués
data = data.T.drop_duplicates()
data = data.T
print ('nombre de colonne duppliquées après supressions des colonne dupliquées est : ', data.T.duplicated().sum())

In [ ]:
# Nombre de lignes duppliquées
print ('Le nombre de lignes duppliquées : ', data.duplicated().sum())

In [ ]:
#Suppression des lignes duppliquées
data = data.drop_duplicates()
print ('Nombre de lignes duppliquées aprés supression des lignes duppliquées : ', data.duplicated().sum())

In [ ]:
print ('Taille de la data après nettoyage des données dupliquées', data.shape)

In [ ]:
#data.to_excel('infront_data/Data_Infront.xlsx', index = False)

#### **Null values handling**

In [ ]:
#Fonction pourcentage_valeurs_nulles input : (data) -----------output-----------> le pourcentage de valeurs nulles dans la data
def pourcentage_valeurs_nulles (df):
    nval = df.shape[0]*df.shape[1]
    nnull = df.isnull().sum().sum()
    return nnull*100 / nval 

In [ ]:
#Le nombre de valeurs nulles dans 'data' 
print ( pourcentage_valeurs_nulles(data), '%  de valeurs de <data> sont des valeurs nulles')

In [ ]:
#Pour détecter les colonnes à supprimer:
#Visualisation le pourcentage de valeurs nulles colonnes par colonnes
pourcentage_na_colonne = data.isnull().sum() * 100 / data.shape[0]
fig2 = px.bar ( x = data.columns, y = pourcentage_na_colonne, width = 2000, height = 600)
fig2.update_layout( title = '% de valeurs nulles par colonne' , xaxis_title = 'Colonne', yaxis_title = 'Pourcentage de valeurs nulles' )
fig2.show()

In [ ]:
#Fonction visualiser_na (data) ------------output----------> heatmap des valeurs nulles de data
def visualiser_na(data):
    fig = px.imshow(data.isnull(), color_continuous_scale = 'plasma', width = 900, height = 700)
    fig.show()

In [ ]:
#Visualisation du heatmap des valeurs nulles
fig1 = px.imshow(data.isnull(), color_continuous_scale = 'plasma', width = 2000, height = 700, labels = {"x": 'Colonnes', "y": 'Lingnes'} )
fig1.update_layout ( title = 'Distribution des valeurs nulles dans la data' )
fig1.show()

In [ ]:
pio.write_image (fig1, 'Valeurs nulles distribution.png' )

In [ ]:
#suppression de tout les lignes contenant des valeurs nulles
data_test = data.dropna (axis = 0)
print ('Nombre de valeurs nulles après suppression de tout les valeurs nulles :', data_test.isnull().sum().sum())
print ('Si on supprime tout les lignes contenant des valeurs nulles, il nous resterait que ', data_test.shape[0], 'Lignes')


In [ ]:
#Supprimons les colonnes contenant plus de pourcent de valeurs nulles
pourcent = 35
colonnes_a_supprimer = data.isnull().sum()*100/data.shape[0]
colonnes_a_supprimer = colonnes_a_supprimer[colonnes_a_supprimer >= pourcent]
print ('Nombre de colonnes à supprimées = ', colonnes_a_supprimer.count())

In [ ]:
#Suppression des colonnes contenant plus de pourcent de valeurs nulles
data_V1 = data.drop (colonnes_a_supprimer.index, axis = 1)
print ('Nombre de colonnes supprimer : ', colonnes_a_supprimer.count())
print ('Taille de data_V1 après suppression des colonnes contenant plus de' ,pourcent , '% de valeurs nulles', data_V1.shape)

In [ ]:
data_V1.columns

In [ ]:
#Supprimons les lignes contenant plus de pourcent_lign de valeurs nulles 
pourcent_ligne = 30
pourcentage_na_ligne = data_V1.T.isnull().sum()*100/data_V1.shape[1]
ligne_a_supprimer = pourcentage_na_ligne[pourcentage_na_ligne >= pourcent_ligne]
print ('Nombre de lignes à supprimées = ', ligne_a_supprimer.count())

|!| 
--- 

30% de valeurs nulles est changeable

In [ ]:
#Suppressions de ces lignes
data_V2 = data_V1.drop( ligne_a_supprimer.index, axis = 0 )
print ('Nombre de lignes a supprimer :', ligne_a_supprimer.count() )
print ('Taille de data_V2 après suppression des lignes contenant plus de ',pourcent_ligne,' de valeurs nulles', data_V2.shape)

In [ ]:
#Suppression de la colonnes 'ISIN'
data_V3 = data_V2.drop(['ISIN'], axis = 1)
print ('taille de data_V3', data_V3.shape)

In [ ]:
data_V3 = data_V2

In [ ]:
#Visualiser les valeurs nulles de data_V3
fig3 = px.imshow(data_V3.isnull(), color_continuous_scale = 'plasma', width = 1500, height = 700)
fig3.show()

In [ ]:
#Supressions des lignes ou 'Sector' est nulle parcequ'on ne peut pas remplacer ces lignes
ligne_sector_a_supprimer = data_V3 [ data_V3['Sector'].isnull()].index
data_V4 = data_V3.drop(ligne_sector_a_supprimer, axis = 0)
print ('taille de data_V3 après suppression des entreprises dont on connait pas le secteur est', data_V4.shape )

In [ ]:
#Remplacons les valeurs 'N/M', 'N/M+', 'N/M-' par des valeurs nulles 
data_V5 = data_V4.replace ( ['N/M', 'N/M+', 'N/M-'], np.nan )
print (' Pourcentage na data_V4:', pourcentage_valeurs_nulles(data_V4))
print (' Pourcentage na data_V5:', pourcentage_valeurs_nulles(data_V5))


In [ ]:
#Resupression des lignes contenant plus de 20% de valeurs nulles 
pourcentage_valeurs_nulles_ligne = data_V5.T.isnull().sum()*100/data_V5.shape[1]
lignes_a_supprimer =  pourcentage_valeurs_nulles_ligne [pourcentage_valeurs_nulles_ligne > 30]
data_V6 = data_V5.drop( lignes_a_supprimer.index, axis = 0 )
print ('Nombre de lignes a supprimer :', len(lignes_a_supprimer) )
print ('taille de data_V6 :', data_V6.shape)

In [ ]:
#suppression de tout les lignes contenant des valeurs nulles
data_test = data_V6.dropna (axis = 0)
print ('Nombre de valeurs nulles après suppression de tout les valeurs nulles :', data_test.isnull().sum().sum())
print ('Si on supprime tout les lignes contenant des valeurs nulles, il nous resterait que ', data_test.shape[0], 'Lignes')

|!|
---

`Dans un premièr temps nous allons supprimer tout les valeurs nulles (Tout ligne contenant une valeur nulle va etre supprimer), et appliquer le clustering
Puis dans un deuxième temps nous allons les remplacer d'autre valeurs.`

|!|
---

In [ ]:
#Supression de tout les valeurs nulles
data_V7 = data_V6.dropna (axis = 0)
print ('Taille de data_V7 : ', data_V7.shape)
print ('Pourcentage de valeur nulle dans data_V7 : ', pourcentage_valeurs_nulles(data_V7))

In [ ]:
data_V7.columns

In [ ]:
#Extraction de la data après suppression de tout les valeurs nulles
data_V7.to_excel ( 'Data Finpack V2/Data_Finpack_V2_sans_na.xlsx', index = True )

In [ ]:
#Suppression de data_V7 de la mémoire 
#del data_V7